/v3/finance/transaction/totals — получите детальную информацию по итоговым суммам транзакций за указанный период:

https://docs.ozon.ru/api/seller/#section/Poluchite-analiticheskie-otchyoty

https://docs.ozon.ru/api/seller/#operation/FinanceAPI_FinanceTransactionTotalV3

Описание метода: https://docs.ozon.ru/api/seller/#operation/AnalyticsAPI_AnalyticsGetData

In [ ]:
import requests
import pandas as pd
## Подключение гугл диска
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def analytics_data_day(date_from,date_to,id,key): #Данные по дням по продавцу, период не более года
    url = "https://api-seller.ozon.ru/v1/analytics/data"

    headers = {
        "Client-Id": id,
        "Api-Key": key,
        "Content-Type": "application/json"
    }

    payload = {
        "date_from": date_from,
        "date_to": date_to,
        "metrics": ["hits_view_search", # показы в поиске и в категории
                    "hits_view_pdp",    # показы на карточке товара
                    "hits_tocart",      # всего добавлено в корзину
                    "session_view",     # всего сессий
                    "conv_tocart",      # общая конверсия в корзину
                    "revenue",          # заказано на сумму
                    "ordered_units",    # заказано товаров
                    "cancellations",    # отменено товаров,
                    "delivered_units",  # доставлено товаров
                    "returns",          # возвращено товаров
                    "adv_view_search_category", # показы в поиске и в категории, спонсорские товары,
                    "adv_view_all",     # показы всего, спонсорские товары,
                    "adv_sum_all",      # всего расходов на рекламу
                    "position_category" # позиция в поиске и категории
                    ],
        "dimension": ["day"],
        "filters": [],
        "sort": [ ],
        "limit": 1000,
        "offset": 0
    }
    # Создаем пустой датафрейм
    df = pd.DataFrame()

    # Пример POST-запроса к API OZON
    response = requests.post(url, headers=headers, json=payload)

    # Проверяем статус-код ответа
    if response.status_code == 200:
        print("analytics_data_day","Client_Id:",id,date_from,date_to,"Запрос успешно выполнен!")

        # Преобразуем ответ в формат JSON
        data = response.json()

        # Извлекаем нужные данные из ответа
        result_data = data["result"]["data"]

        # Добавляем данные в датафрейм
        for item in result_data:
            metrics = item["metrics"]
            row={'id':item["dimensions"][0]['id']}
            row.update({"client_id":                   id})
            row.update({"hits_view_search":            metrics[0]})
            row.update({"hits_view_pdp":               metrics[1]})
            row.update({"hits_tocart":                 metrics[2]})
            row.update({"session_view":                metrics[3]})
            row.update({"conv_tocart":                 metrics[4]})
            row.update({"revenue":                     metrics[5]})
            row.update({"ordered_units":               metrics[6]})
            row.update({"cancellations":               metrics[7]})
            row.update({"delivered_units":             metrics[8]})
            row.update({"returns":                     metrics[9]})
            row.update({"adv_view_search_category":    metrics[10]})
            row.update({"adv_view_all":                metrics[11]})
            row.update({"adv_sum_all":                 metrics[12]})
            row.update({"position_category":           metrics[13]})
            df = pd.concat([df, pd.DataFrame([row])])

        # Сбрасываем индексы датафрейма
        df.reset_index(drop=True, inplace=True)

    else:
        print("Ошибка при выполнении запроса. Код статуса:", response.status_code)
        print(response.text)
    return df

In [ ]:
def rating_day(date_from,date_to,id,key): #Рейтинг по дням по продавцу, период не более квартала
    """
    date_from="2023-06-01"
    date_to="2023-06-30"
    id="26652"
    key="6f40a57e-f3eb-4ae4-9943-76e203f3e26e"
    """
    url = "https://api-seller.ozon.ru/v1/rating/history"

    headers = {
        "Client-Id": id,
        "Api-Key": key,
        "Content-Type": "application/json"
    }

    payload = {"date_from": date_from+"T00:00:00Z",
              "date_to": date_to+"T23:59:59Z",
              "ratings": [
                    "rating_review_avg_score_total",       # средняя оценка всех товаров.
                    "rating_price",                        # индекс цен: отношение стоимости ваших товаров к самой низкой цене на такой же товар на других площадках.
                    "rating_order_cancellation",           # процент отмен FBS#отправлений по вашей вине за последние 7 дней. Текущий и предыдущий день не учитываются.
                    "rating_shipment_delay",               # процент FBS#отправлений, которые за последние 7 дней не были переданы в доставку по вашей вине. Текущий и предыдущий день не учитываются.
                    "rating_ssl"                          # оценка работы по FBO. Учитывает rating_on_time_supply_delivery, rating_on_time_supply_cancellation и rating_order_accuracy.
              ],
              "with_premium_scores": True
    }
    '''
                    "rating_on_time",                      # процент заказов, выполненных вовремя за последние 30 дней.
                    "rating_review_avg_score_total",       # средняя оценка всех товаров.
                    "rating_price",                        # индекс цен: отношение стоимости ваших товаров к самой низкой цене на такой же товар на других площадках.
                    "rating_order_cancellation",           # процент отмен FBS#отправлений по вашей вине за последние 7 дней. Текущий и предыдущий день не учитываются.
                    "rating_shipment_delay",               # процент FBS#отправлений, которые за последние 7 дней не были переданы в доставку по вашей вине. Текущий и предыдущий день не учитываются.
                    "rating_ssl",                          # оценка работы по FBO. Учитывает rating_on_time_supply_delivery, rating_on_time_supply_cancellation и rating_order_accuracy.
                    "rating_on_time_supply_delivery",      # процент поставок, которые вы привезли на склад в выбранный временной интервал за последние 60 дней.
                    "rating_order_accuracy",               # процент поставок без излишков, недостач, пересорта и брака за последние 60 дней.
                    "rating_on_time_supply_cancellation",  # процент заявок на поставку, которые завершились или были отменены без опоздания за последние 60 дней.
                    "rating_reaction_time",                # время, в течение которого покупатели в среднем ждали ответа на своё первое сообщение в чате за последние 30 дней.
                    "rating_average_response_time",        # время, в течение которого покупатели в среднем ждали вашего ответа за последние 30 дней.
                    "rating_replied_dialogs_ratio"         # доля диалогов хотя бы с одним вашим ответом в течение 24 часов за последние 30 дней.
    '''
    # Создаем пустой датафрейм
    df = pd.DataFrame()

    # Пример POST-запроса к API OZON
    response = requests.post(url, headers=headers, json=payload)

    # Проверяем статус-код ответа
    if response.status_code == 200:
        print("rating_day","Client_Id:",id,date_from,date_to,"Запрос успешно выполнен!")

        # Преобразуем ответ в формат JSON
        data = response.json()

        # Извлекаем нужные данные из ответа
        result_data = data["ratings"]
        # Добавляем данные в датафрейм
        for i in range(len(result_data[0]['values'])): # по датам
          t_date=result_data[0]['values'][i]['date_from'][:10]
          row={'id':t_date}
          row.update({"client_id":    id})
          for j in range(len(result_data)): # по количеству фич
              row.update({result_data[j]['rating']:result_data[j]['values'][i]['value']})
          df = pd.concat([df, pd.DataFrame([row])])

        # Сбрасываем индексы датафрейма
        df.reset_index(drop=True, inplace=True)

    else:
        print("Ошибка при выполнении запроса. Код статуса:", response.status_code)
        print(response.text)
    return df

In [ ]:
# на будующее выгрузка клиентов и ключей из файла и дополнение базы по запросу
keys = pd.read_csv('/content/gdrive/MyDrive/marketplace/db/keys.csv')
print(keys.shape)
for i in range(keys.shape[0]):
    id=keys.iloc[i,0]
    key=keys.iloc[i,1]
    print(id,key)

(2, 2)
26652 6f40a57e-f3eb-4ae4-9943-76e203f3e26e
142461 ae7efb74-8613-4936-8e9a-55e344b8c109


In [ ]:
id=['26652','142461'] #
key=['6f40a57e-f3eb-4ae4-9943-76e203f3e26e','ae7efb74-8613-4936-8e9a-55e344b8c109'] #
y=2023
m=6
y_e=2023
m_e=6
# Создаем пустой датафрейм
day_table = pd.DataFrame()

while(y<=y_e and (y==y_e and m<=m_e) or (y<y_e and m>m_e)):
    d1=f'{y}-{m:02}-02'
    m+=1
    if m>12:
        m=1
        y+=1
    d2=f'{y}-{m:02}-01'
    for i in range(len(id)):
        data_table=analytics_data_day(d1,d2,id[i],key[i])
        rating_table=rating_day(d1,d2,id[i],key[i])
        day_table1=pd.merge(data_table,rating_table,how='left',on=['id','client_id'])
        day_table=pd.concat([day_table,day_table1], axis=0, sort=False)
    print(d1,d2)

analytics_data_day Client_Id: 26652 2023-06-02 2023-07-01 Запрос успешно выполнен!
rating_day Client_Id: 26652 2023-06-02 2023-07-01 Запрос успешно выполнен!
analytics_data_day Client_Id: 142461 2023-06-02 2023-07-01 Запрос успешно выполнен!
rating_day Client_Id: 142461 2023-06-02 2023-07-01 Запрос успешно выполнен!
2023-06-02 2023-07-01


In [ ]:
display(day_table)

In [ ]:
day_table.to_csv('/content/gdrive/MyDrive/marketplace/db/days.csv', index=False)


In [ ]:
import os
directory = "/content/gdrive/MyDrive/marketplace/db/"
# Создаем пустой датафрейм
day_table = pd.DataFrame()
for root, dirs, files in os.walk(directory):
    for file in files:
        if file[:4]=='days':
            print("*" * 60)
            path = os.path.join(root, file)
            print(path)
            table = pd.read_csv(path)
            day_table=pd.concat([day_table,table], axis=0, sort=False)

************************************************************
/content/gdrive/MyDrive/marketplace/db/days1.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days2.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days3.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days4.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days5.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days6.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days7.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days8.csv
************************************************************
/content/gdrive/MyDrive/marketplace/db/days9.csv
